In [30]:
import csv
import pandas as pd
import numpy as np
import torch



#data = Data(x = embeddings_features,edge_index=t.t().contiguous())

In [15]:
import networkx as nx

In [17]:
# Nodes: 5242 Edges: 28980
G = nx.read_edgelist("data/ca-GrQc.txt")

In [21]:
betweenness= nx.betweenness_centrality(G)

In [25]:
degree = nx.degree_centrality(G)

In [27]:
eigenvector = nx.eigenvector_centrality(G)

In [28]:
closeness = nx.closeness_centrality(G)

In [39]:
df = pd.read_csv("data/ca-GrQc.txt", header=None,sep="\t",skiprows=4)

In [48]:
len(degree)

5242

In [49]:
# One-hot: untested, tested positive, tested negative, positive-in-the-past
dynamic = np.zeros((len(degree),4))

In [64]:
keys=[int(n) for n in list(betweenness.keys())]

In [66]:
sorted_keys = sorted(keys)

In [72]:
betweenness_vec=[]
closeness_vec=[]
eigenvector_vec=[]
degree_vec=[]
for n in sorted_keys:
    betweenness_vec.append(betweenness[str(n)])
    closeness_vec.append(closeness[str(n)])
    eigenvector_vec.append(eigenvector[str(n)])
    degree_vec.append(degree[str(n)])

In [77]:
betweenness_vec = np.asarray(betweenness_vec)
closeness_vec = np.asarray(closeness_vec)
eigenvector_vec = np.asarray(eigenvector_vec)
degree_vec = np.asarray(degree_vec)

In [90]:
static = np.stack((betweenness_vec,closeness_vec,eigenvector_vec,degree_vec),axis=1)

In [91]:
static.shape

(5242, 4)

In [101]:
node_features = np.concatenate((static,dynamic),axis=1)

In [103]:
t=25

### Mapping

In [104]:
edges =df.values

In [105]:
edges.shape

(28980, 2)

In [179]:
all_nodes = np.unique(edges.T[0])   # We just need to find all unique occurences in one column only since the matrix contains undirected entries (each edge appears twice in reversed direction)
all_nodes.shape

edge_index = np.zeros(edges.shape)

dict_map = {k: v for v, k in enumerate(all_nodes)}

for i in range(edge_index.shape[0]):
    for j in range(edge_index.shape[1]):
        edge_index[i,j] = dict_map[edges[i,j]]



In [107]:
from numpy.random import default_rng


In [147]:
# t: how many time steps
# num_edges: how many edges in the network
# subset_size: how big the subset to pick from total edges
def sample(t, edges, subset_size):
    num_edges=len(edges)
    t_edges = np.zeros((t,int(subset_size*num_edges),3))
    rng = default_rng()
    for i in range(t):
        vals = rng.integers(low=0, high=len(edges)-1, size=int(num_edges*subset_size))
        edges_sub = edges[vals]
        vals_p = rng.uniform(low = 0.5, high = 1.0, size = int(num_edges*subset_size))
        vals_p = vals_p.reshape((-1, 1))
        combined = np.concatenate((edges_sub,vals_p),axis=1)
        t_edges[i]=combined
    return t_edges
    

In [182]:
t_edges= sample(10,edge_index,0.1)

In [183]:
t_edges.shape

(10, 2898, 3)

In [151]:
class Local_Diffusion(torch.nn.Module):
    def __init__(self,data):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 64)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        result = self.conv1(x, edge_index,edge_weight = data.edge_attr)
        result = F.relu(result)
        return result

In [184]:
t_edges_index = t_edges[:,:,0:2]

In [185]:
t_edges_index= t_edges_index.astype(int)

In [186]:
t = torch.from_numpy(edge_index)

In [187]:
t_edges_attr =  t_edges[:,:,2]

In [205]:
import sys

In [206]:
sys.path.append("/usr/bin/gcc")

In [207]:
pip install torch_sparse

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 43 kB 6.8 MB/s             
  Preparing metadata (setup.py) ... done
  ERROR: Command errored out with exit status 1:
   command: /share/pkg.7/python3/3.8.10/install/bin/python3.8 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/scratch/1142539.1.academic-gpu/pip-install-uhhu6ggl/torch-sparse_3c4d9f2415c1426d9b99f4f29c7cefa7/setup.py'"'"'; __file__='"'"'/scratch/1142539.1.academic-gpu/pip-install-uhhu6ggl/torch-sparse_3c4d9f2415c1426d9b99f4f29c7cefa7/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /scratch/1142539.1.academic-gpu/pip-wheel-ifj62lyo
       cwd: /scratch/1142539.1.academic-gpu/pip-install-uhhu6ggl/tor

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [201]:
pip install --verbose --no-cache-dir torch-sparse

Using pip 21.3 from /usr3/graduate/zche/.local/lib/python3.8/site-packages/pip (python 3.8)
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 43 kB 7.0 MB/s             
  Running command python setup.py egg_info
  running egg_info
  creating /scratch/1142539.1.academic-gpu/pip-pip-egg-info-77sgqoo9/torch_sparse.egg-info
  writing /scratch/1142539.1.academic-gpu/pip-pip-egg-info-77sgqoo9/torch_sparse.egg-info/PKG-INFO
  writing dependency_links to /scratch/1142539.1.academic-gpu/pip-pip-egg-info-77sgqoo9/torch_sparse.egg-info/dependency_links.txt
  writing requirements to /scratch/1142539.1.academic-gpu/pip-pip-egg-info-77sgqoo9/torch_sparse.egg-info/requires.txt
  writing top-level names to /scratch/1142539.1.academic-gpu/pip-pip-egg-info-77sgqoo9/torch_sparse.egg-info/top_level.txt
  writing manifest file '/scratch/1142539.1.academic-gpu/pip-pip-egg-info-77sgqoo9/torch_sparse.egg-info/SOURCES.txt'
  reading manifest 

    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.8
    creating build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/spspmm.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/sample.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/masked_select.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/__init__.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/rw.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/cat.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/matmul.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/padding.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/spmm.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/utils.py -> build/lib.linux-x86_64-3.8/torch_sparse
    copying torch_sparse/convert.py -> build/lib.

In [194]:
pip install torch_geometric

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [199]:
torch

<module 'torch' from '/share/pkg.7/pytorch/1.9.0/install/lib/python3.8/site-packages/torch/__init__.py'>

In [196]:
from torch_geometric.data import Data

ModuleNotFoundError: No module named 'torch_sparse'

In [188]:
data = Data(x = node_features, edge_index=t.t().contiguous(),edge_attr= t_edges_attr)

NameError: name 'Data' is not defined

In [ ]:
with open('node_features.npy', 'wb') as f:
    np.save(f, node_features)

In [ ]:
with open('node_t_edge_example.npy', 'wb') as f:
    np.save(f, t_edges_index)